In [4]:
import numpy as np
from pyAudioAnalysis import audioBasicIO
import Audio_Feature_Extraction as AFE
from pyAudioAnalysis import audioFeatureExtraction
from sklearn import mixture
from sklearn.externals import joblib
import glob

In [19]:
[Fs, x] = audioBasicIO.readAudioFile("03-01-05-01-01-01-01.wav");
window_size = 0.030
overlap = 0.015
VTH_Multiplier = 0.05
VTH_range=100
n_mixtures = 16
max_iterations = 75
calc_deltas=False

48000

In [25]:
energy = [ s**2 for s in x]
Voiced_Threshold = VTH_Multiplier*np.mean(energy)
clean_samples=[0]

for sample_set in xrange(0,len(x)-VTH_range,VTH_range):
    sample_set_th = np.mean(energy[sample_set:sample_set+VTH_range])
    if sample_set_th>Voiced_Threshold:
        clean_samples.extend(x[sample_set:sample_set+VTH_range])

energy=None

MFCC13_F = AFE.stFeatureExtraction(clean_samples, Fs, Fs*window_size, Fs*overlap)

MFCC13_F.shape


(95, 13)

In [26]:
delta_MFCC = np.zeros(MFCC13_F.shape)
for t in range(delta_MFCC.shape[1]):
    index_t_minus_one,index_t_plus_one=t-1,t+1

    if index_t_minus_one<0:    
        index_t_minus_one=0
    if index_t_plus_one>=delta_MFCC.shape[1]:
        index_t_plus_one=delta_MFCC.shape[1]-1

    delta_MFCC[:,t]=0.5*(MFCC13_F[:,index_t_plus_one]-MFCC13_F[:,index_t_minus_one] )

In [27]:
double_delta_MFCC = np.zeros(MFCC13_F.shape)
for t in range(double_delta_MFCC.shape[1]):

    index_t_minus_one,index_t_plus_one, index_t_plus_two,index_t_minus_two=t-1,t+1,t+2,t-2

    if index_t_minus_one<0:
        index_t_minus_one=0
    if index_t_plus_one>=delta_MFCC.shape[1]:
        index_t_plus_one=delta_MFCC.shape[1]-1
    if index_t_minus_two<0:
        index_t_minus_two=0
    if index_t_plus_two>=delta_MFCC.shape[1]:
        index_t_plus_two=delta_MFCC.shape[1]-1

    double_delta_MFCC[:,t]=0.1*( 2*MFCC13_F[:,index_t_plus_two]+MFCC13_F[:,index_t_plus_one]
                                -MFCC13_F[:,index_t_minus_one]-2*MFCC13_F[:,index_t_minus_two] )

In [28]:
Combined_MFCC_F = np.concatenate((MFCC13_F,delta_MFCC,double_delta_MFCC),axis=1)
Combined_MFCC_F.shape

(95, 39)

In [29]:
gmm = mixture.GaussianMixture(n_components=n_mixtures, covariance_type='diag', 
                              max_iter = max_iterations ).fit(Combined_MFCC_F)
joblib.dump(gmm, 'be_mix.pkl') 

['be_mix.pkl']